# Imports

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
import copy
import geodata
import time
import math

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Settings

In [3]:
scenario = "2030"
charging_power = 11
cluster = 10

# Reading Inputs

In [4]:
with open('data/ev_population.pickle', 'rb') as f:
    ev_population = pickle.load(f)

In [5]:
ev_population.head(3)

,PLZ_Gemeinde,KKZ,2020,2025,2030,2035,2040,2045,2050,Gemeinde,PLZ,Typ,Typ_differenziert,Profil,NUTS3
54298,54298 AACH,7211,7.838229,68.057259,188.480445,349.229940,478.115967,545.521333,573.087811,AACH,54298,Landgemeinde,Landgemeinde,[HT6],DEB25
78267,"78267 AACH,ST.",8335,23.522947,177.387925,480.778145,877.060477,1187.826308,1347.949286,1412.966666,AACH,78267,Größere Kleinstadt,Größere Kleinstadt,[HT5],DE138
52062,"52062 AACHEN, STADT",5334,1577.645762,11648.828673,31459.283476,57240.093137,77382.428026,87735.184826,91933.922371,AACHEN,52062,Großstadt,Kleinere Großstadt,"[HT1, HT2]",DEA2D


In [6]:
# Define the file path
file_path = "output/export_sc" + scenario  + "_cp" + str(charging_power) + "_cluster" + str(cluster) + ".xlsx"

# Read the DataFrames from different sheets
profiles_all = pd.read_excel(file_path, sheet_name='load', index_col=0, header=[0,1,2,3])
pos_flex = pd.read_excel(file_path, sheet_name='pos_flex', index_col=0, header=[0,1,2,3])
neg_flex = pd.read_excel(file_path, sheet_name='neg_flex', index_col=0, header=[0,1,2,3])
driving = pd.read_excel(file_path, sheet_name='driving', index_col=0, header=[0,1,2,3])

## Profile Generation

In [7]:
dict_charging_location_new = {}
dict_share_at_home = {}
dict_share_at_home["Landgemeinde"]         = .90
dict_share_at_home["Kleine Kleinstadt"]    = .87
dict_share_at_home["Größere Kleinstadt"]   = .83
dict_share_at_home["Kleinere Mittelstadt"] = .72
dict_share_at_home["Größere Mittelstadt"]  = .78
dict_share_at_home["Kleinere Großstadt"]   = .62
dict_share_at_home["Große Großstadt"]      = .42

In [8]:
for loc in dict_share_at_home:
    dict_charging_location_new[loc] = {}
    for charge in ["Arbeitsplatz_Öffentlich","Arbeitsplatz_Zuhause","Zuhause","Arbeitsplatz_Zuhause_Öffentlich","Arbeitsplatz","Öffentlich","Zuhause_Öffentlich"]:
        if charge in ["Arbeitsplatz","Öffentlich","Arbeitsplatz_Öffentlich"]: 
            dict_charging_location_new[loc][charge] = (1-dict_share_at_home[loc])/3
        else: 
            dict_charging_location_new[loc][charge] = (dict_share_at_home[loc])/4  

In [9]:
generate = True

In [10]:
if generate:
    start = time.time()
     
    # dictionary for each region
    regions_profiles = {}
    regions_neg_flex = {}
    regions_pos_flex = {}
    regions_driving = {}

    # define considered years
    years = [int(scenario)]

    # iterate each region
    for region in ev_population.index:
        regions_profiles[region] = {}
        regions_neg_flex[region] = {}
        regions_pos_flex[region] = {}
        regions_driving[region] = {}
        # iterate each year
        for year in years:
            regions_profiles[region][year] = {}
            regions_neg_flex[region][year] = {}
            regions_pos_flex[region][year] = {}
            regions_driving[region][year] = {}
            # iterate day of week
            for day in ["Wochentag","Wochenende"]:
                # initialize empty container for each region and time series
                regions_profiles[region][year][day]            = profiles_all.loc[:,(ev_population.loc[region,"Typ_differenziert"],day,"Arbeitsplatz_Zuhause_Öffentlich")].copy()
                regions_neg_flex[region][year][day]            = neg_flex.loc[:,(ev_population.loc[region,"Typ_differenziert"],day,"Arbeitsplatz_Zuhause_Öffentlich")].copy()
                regions_pos_flex[region][year][day]            = pos_flex.loc[:,(ev_population.loc[region,"Typ_differenziert"],day,"Arbeitsplatz_Zuhause_Öffentlich")].copy()
                regions_driving[region][year][day]             = driving.loc[:,(ev_population.loc[region,"Typ_differenziert"],day,"Arbeitsplatz_Zuhause_Öffentlich")].copy()
                regions_profiles[region][year][day].loc[:,:]   = 0
                regions_neg_flex[region][year][day].loc[:,:]   = 0
                regions_pos_flex[region][year][day].loc[:,:]   = 0
                regions_driving[region][year][day].loc[:,:]    = 0
                
                # sum by all charging location
                for charging_location in dict_charging_location_new[ev_population.loc[region,"Typ_differenziert"]]:
                    # add profile to container
                    regions_profiles[region][year][day] +=     profiles_all.loc[:,(ev_population.loc[region,"Typ_differenziert"],day,charging_location)] * ev_population.loc[region,year] * dict_charging_location_new[ev_population.loc[region,"Typ_differenziert"]][charging_location] 
                    regions_driving[region][year][day]  +=     driving.loc[:,(ev_population.loc[region,"Typ_differenziert"],day,charging_location)] * ev_population.loc[region,year] * dict_charging_location_new[ev_population.loc[region,"Typ_differenziert"]][charging_location] 
                
                # sum only by charging location with home charging
                for charging_location in pos_flex.columns.get_level_values(2).drop_duplicates():     
                    regions_pos_flex[region][year][day] +=     pos_flex.loc[:,(ev_population.loc[region,"Typ_differenziert"],day,charging_location)] * ev_population.loc[region,year] * dict_charging_location_new[ev_population.loc[region,"Typ_differenziert"]][charging_location] 
                    regions_neg_flex[region][year][day] +=     neg_flex.loc[:,(ev_population.loc[region,"Typ_differenziert"],day,charging_location)] * ev_population.loc[region,year] * dict_charging_location_new[ev_population.loc[region,"Typ_differenziert"]][charging_location] 
    
    with open('data/regions_profiles_{}.pickle'.format(scenario), 'wb') as f:
        pickle.dump(regions_profiles, f)
    with open('data/regions_pos_flex_{}.pickle'.format(scenario), 'wb') as f:
        pickle.dump(regions_pos_flex, f)
    with open('data/regions_neg_flex_{}.pickle'.format(scenario), 'wb') as f:
        pickle.dump(regions_neg_flex, f)
    with open('data/regions_driving_{}.pickle'.format(scenario), 'wb') as f:
        pickle.dump(regions_driving, f)
    
    # measure runtime        
    end = time.time()
    print("Elapsed Time: ", end - start)
    
else:
    with open('data/regions_profiles_{}.pickle'.format(scenario), 'rb') as f:
        regions_profiles = pickle.load(f)
    with open('data/regions_pos_flex_{}.pickle'.format(scenario), 'rb') as f:
        regions_pos_flex = pickle.load(f)
    with open('data/regions_neg_flex_{}.pickle'.format(scenario), 'rb') as f:
        regions_neg_flex = pickle.load(f)
    with open('data/regions_driving_{}.pickle'.format(scenario), 'rb') as f:
        regions_driving = pickle.load(f)
    

D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:925: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


Elapsed Time:  1403.3924469947815


## Relabel Regions (PLZ to NUTS3)

In [11]:
relabel = True

In [12]:
if relabel:
    start = time.time()
    
    #change labels
    nuts_to_plz = pd.read_csv("data//nuts3ToPlz.csv",delimiter=";")

    nuts_dict_profiles = {}
    nuts_dict_pos_flex = {}
    nuts_dict_neg_flex = {}
    nuts_dict_driving = {}
    print("The following regions were not found in the nuts_to_plz:")
    for region in ev_population.index:
        if np.any(nuts_to_plz[nuts_to_plz["CODE"]==int(region[0])]["NUTS3"].values): 
            ev_population.loc[region[0],"NUTS3"] = nuts_to_plz.loc[nuts_to_plz["CODE"]==int(region[0]),"NUTS3"].values[0]
            
            if nuts_to_plz[nuts_to_plz["CODE"]==int(region[0])]["NUTS3"].values[0] not in nuts_dict_profiles.keys():
                nuts_dict_profiles[nuts_to_plz[nuts_to_plz["CODE"]==int(region[0])]["NUTS3"].values[0]] = {}
                nuts_dict_profiles[nuts_to_plz[nuts_to_plz["CODE"]==int(region[0])]["NUTS3"].values[0]]["list"] = []
                nuts_dict_pos_flex[nuts_to_plz[nuts_to_plz["CODE"]==int(region[0])]["NUTS3"].values[0]] = {}
                nuts_dict_pos_flex[nuts_to_plz[nuts_to_plz["CODE"]==int(region[0])]["NUTS3"].values[0]]["list"] = []
                nuts_dict_neg_flex[nuts_to_plz[nuts_to_plz["CODE"]==int(region[0])]["NUTS3"].values[0]] = {}
                nuts_dict_neg_flex[nuts_to_plz[nuts_to_plz["CODE"]==int(region[0])]["NUTS3"].values[0]]["list"] = []
                nuts_dict_driving[nuts_to_plz[nuts_to_plz["CODE"]==int(region[0])]["NUTS3"].values[0]] = {}
                nuts_dict_driving[nuts_to_plz[nuts_to_plz["CODE"]==int(region[0])]["NUTS3"].values[0]]["list"] = []
            
            nuts_dict_profiles[nuts_to_plz[nuts_to_plz["CODE"]==int(region[0])]["NUTS3"].values[0]]["list"].append(regions_profiles[region])            
            nuts_dict_pos_flex[nuts_to_plz[nuts_to_plz["CODE"]==int(region[0])]["NUTS3"].values[0]]["list"].append(regions_pos_flex[region])
            nuts_dict_neg_flex[nuts_to_plz[nuts_to_plz["CODE"]==int(region[0])]["NUTS3"].values[0]]["list"].append(regions_neg_flex[region])
            nuts_dict_driving[nuts_to_plz[nuts_to_plz["CODE"]==int(region[0])]["NUTS3"].values[0]]["list"].append(regions_driving[region])
        else: print(region)
            
    for region in nuts_dict_profiles:
        nuts_dict_profiles[region][int(scenario)] = {}                                                                                                          
        nuts_dict_pos_flex[region][int(scenario)] = {}
        nuts_dict_neg_flex[region][int(scenario)] = {}
        nuts_dict_driving[region][int(scenario)] = {}
        for day in ["Wochentag","Wochenende"]:
            nuts_dict_profiles[region][int(scenario)][day] = sum([i[int(scenario)][day] for i in nuts_dict_profiles[region]["list"]])
            nuts_dict_pos_flex[region][int(scenario)][day] = sum([i[int(scenario)][day] for i in nuts_dict_pos_flex[region]["list"]])
            nuts_dict_neg_flex[region][int(scenario)][day] = sum([i[int(scenario)][day] for i in nuts_dict_neg_flex[region]["list"]])
            nuts_dict_driving[region][int(scenario)][day] = sum([i[int(scenario)][day] for i in nuts_dict_driving[region]["list"]])
            
        nuts_dict_profiles[region].pop('list', None)
        nuts_dict_pos_flex[region].pop('list', None)
        nuts_dict_neg_flex[region].pop('list', None)
        nuts_dict_driving[region].pop('list', None)
        
    # The sum of the charging profiles for a day must be equal to the sum of the driving profile for that day
    for key1 in nuts_dict_driving.keys():
        for key2 in  nuts_dict_driving[key1].keys():
            for key3 in  nuts_dict_driving[key1][key2].keys():
                for c in nuts_dict_driving[key1][key2][key3].columns:
                    fac = sum(nuts_dict_profiles[key1][key2][key3].loc[:,c]) / sum(nuts_dict_driving[key1][key2][key3].loc[:,c])
                    if math.isnan(fac):
                        fac = 1
                    nuts_dict_driving[key1][key2][key3].loc[:,c] = nuts_dict_driving[key1][key2][key3].loc[:,c] * fac    
                    
    with open('data/nuts_profiles_{}.pickle'.format(scenario), 'wb') as f:
        pickle.dump(nuts_dict_profiles, f)                                                                                                              
    with open('data/nuts_pos_flex_{}.pickle'.format(scenario), 'wb') as f:
        pickle.dump(nuts_dict_pos_flex, f)
    with open('data/nuts_neg_flex_{}.pickle'.format(scenario), 'wb') as f:
        pickle.dump(nuts_dict_neg_flex, f)
    with open('data/ev_population.pickle', 'wb') as f:
        pickle.dump(ev_population, f)
    with open('data/nuts_driving_{}.pickle'.format(scenario), 'wb') as f:
        pickle.dump(nuts_dict_driving, f)

    # measure runtime        
    end = time.time()
    print("Elapsed Time: ", end - start)
    
else:
    with open('data/nuts_profiles_{}.pickle'.format(scenario), 'rb') as f:
        nuts_dict_profiles = pickle.load(f)
    with open('data/nuts_pos_flex_{}.pickle'.format(scenario), 'rb') as f:
        nuts_dict_pos_flex = pickle.load(f)
    with open('data/nuts_neg_flex_{}.pickle'.format(scenario), 'rb') as f:
        nuts_dict_neg_flex = pickle.load(f)
    with open('data/ev_population.pickle', 'rb') as f:
        ev_population = pickle.load(f)
    with open('data/nuts_driving{}.pickle'.format(scenario), 'rb') as f:
        nuts_dict_driving = pickle.load(f)

The following regions were not found in the nuts_to_plz:
('06922', '06922  ANNABURG,STADT')
('99437', '99437  BAD BERKA,STADT')
('37534', '37534  Bad Grund (Harz)')
('09227', '09227  BURKHARDTSDORF')
('03007', '03007  COTTBUS')
('08439', '08439  CRIMMITSCHAU, STADT')
('06813', '06813  DESSAU-ROßLAU')
('01827', '01827  DUERRROEHRSDORF-DITTERSBACH')
('37547', '37547  EINBECK, STADT')
('04445', '04445  GROSSPOESNA')
('79683', '79683  KLEINES WIESENTAL')
('07558', '07558  KRAFTSDORF')
('31097', '31097  Lamspringe')
('08312', '08312  LAUTER-BERNSBACH, STADT')
('04007', '04007  LEIPZIG')
('09238', '09238  LICHTENAU')
('08146', '08146  LICHTENSTEIN/SA., STADT')
('08399', '08399  LIMBACH-OBERFROHNA, STADT')
('24875', '24875  MITTELANGELN')
('64743', '64743  Oberzent, Stadt')
('02744', '02744  ODERWITZ')
('04460', '04460  PEGAU, STADT')
('04657', '04657  PENIG, STADT')
('82243', '82243  POECKING')
('16921', '16921  PRITZWALK, STADT')
('04448', '04448  RACKWITZ')
('65423', '65423  RUESSELSHEIM,S

## Relabel regions (PLZ to Siedlungstypen)

In [13]:
relabel2 = True
if relabel2:
    start = time.time()
    
    # change labels
    nuts_to_plz = pd.read_csv("data//nuts3ToPlz.csv", delimiter=";")

    siedlungstyp_dict_profiles = {}
    siedlungstyp_dict_pos_flex = {}
    siedlungstyp_dict_neg_flex = {}
    siedlungstyp_dict_driving = {}
    print("The following regions were not found in the nuts_to_plz:")
    
    for region in ev_population.index:
        if np.any(nuts_to_plz[nuts_to_plz["CODE"] == int(region[0])]["NUTS3"].values):
            if ev_population[ev_population["PLZ"] == region[0]]["Typ"].values[0] not in siedlungstyp_dict_profiles.keys():
                siedlungstyp_dict_profiles[ev_population[ev_population["PLZ"] == region[0]]["Typ"].values[0]] = {}
                siedlungstyp_dict_profiles[ev_population[ev_population["PLZ"] == region[0]]["Typ"].values[0]]["list"] = []
                siedlungstyp_dict_pos_flex[ev_population[ev_population["PLZ"] == region[0]]["Typ"].values[0]] = {}
                siedlungstyp_dict_pos_flex[ev_population[ev_population["PLZ"] == region[0]]["Typ"].values[0]]["list"] = []
                siedlungstyp_dict_neg_flex[ev_population[ev_population["PLZ"] == region[0]]["Typ"].values[0]] = {}
                siedlungstyp_dict_neg_flex[ev_population[ev_population["PLZ"] == region[0]]["Typ"].values[0]]["list"] = []
                siedlungstyp_dict_driving[ev_population[ev_population["PLZ"] == region[0]]["Typ"].values[0]] = {}
                siedlungstyp_dict_driving[ev_population[ev_population["PLZ"] == region[0]]["Typ"].values[0]]["list"] = []

            # Profiles according to settlement type
            siedlungstyp_dict_profiles[ev_population[ev_population["PLZ"] == region[0]]["Typ"].values[0]]["list"].append(regions_profiles[region])
            siedlungstyp_dict_pos_flex[ev_population[ev_population["PLZ"] == region[0]]["Typ"].values[0]]["list"].append(regions_pos_flex[region])
            siedlungstyp_dict_neg_flex[ev_population[ev_population["PLZ"] == region[0]]["Typ"].values[0]]["list"].append(regions_neg_flex[region])
            siedlungstyp_dict_driving[ev_population[ev_population["PLZ"] == region[0]]["Typ"].values[0]]["list"].append(regions_driving[region])

        else:
            print(region)
    
    for typ in siedlungstyp_dict_profiles:
        siedlungstyp_dict_profiles[typ][int(scenario)] = {}
        siedlungstyp_dict_pos_flex[typ][int(scenario)] = {}
        siedlungstyp_dict_neg_flex[typ][int(scenario)] = {}
        siedlungstyp_dict_driving[typ][int(scenario)] = {}
        for day in ["Wochentag", "Wochenende"]:
            siedlungstyp_dict_profiles[typ][int(scenario)][day] = sum([i[int(scenario)][day] for i in siedlungstyp_dict_profiles[typ]["list"]])
            siedlungstyp_dict_pos_flex[typ][int(scenario)][day] = sum([i[int(scenario)][day] for i in siedlungstyp_dict_pos_flex[typ]["list"]])
            siedlungstyp_dict_neg_flex[typ][int(scenario)][day] = sum([i[int(scenario)][day] for i in siedlungstyp_dict_neg_flex[typ]["list"]])
            siedlungstyp_dict_driving[typ][int(scenario)][day] = sum([i[int(scenario)][day] for i in siedlungstyp_dict_driving[typ]["list"]])

        siedlungstyp_dict_profiles[typ].pop('list', None)
        siedlungstyp_dict_pos_flex[typ].pop('list', None)
        siedlungstyp_dict_neg_flex[typ].pop('list', None)
        siedlungstyp_dict_driving[typ].pop('list', None)
        
    # The sum of the charging profiles for a day must be equal to the sum of the driving profile for that day
    for key1 in siedlungstyp_dict_driving.keys():
        for key2 in siedlungstyp_dict_driving[key1].keys():
            for key3 in siedlungstyp_dict_driving[key1][key2].keys():
                for c in siedlungstyp_dict_driving[key1][key2][key3].columns:
                    fac = sum(siedlungstyp_dict_profiles[key1][key2][key3].loc[:,c]) / sum(siedlungstyp_dict_driving[key1][key2][key3].loc[:,c])
                    if math.isnan(fac):
                        fac = 1
                    siedlungstyp_dict_driving[key1][key2][key3].loc[:,c] = siedlungstyp_dict_driving[key1][key2][key3].loc[:,c] * fac

    with open('data/siedlungstyp_profiles_{}.pickle'.format(scenario), 'wb') as f:
        pickle.dump(siedlungstyp_dict_profiles, f)
    with open('data/siedlungstyp_pos_flex_{}.pickle'.format(scenario), 'wb') as f:
        pickle.dump(siedlungstyp_dict_pos_flex, f)
    with open('data/siedlungstyp_neg_flex_{}.pickle'.format(scenario), 'wb') as f:
        pickle.dump(siedlungstyp_dict_neg_flex, f)
    with open('data/siedlungstyp_driving_{}.pickle'.format(scenario), 'wb') as f:
        pickle.dump(siedlungstyp_dict_driving, f)

    # measure runtime
    end = time.time()
    print("Elapsed Time: ", end - start)

Folgende Regionen wurden in der nuts_to_plz nicht gefunden:
('06922', '06922  ANNABURG,STADT')
('99437', '99437  BAD BERKA,STADT')
('37534', '37534  Bad Grund (Harz)')
('09227', '09227  BURKHARDTSDORF')
('03007', '03007  COTTBUS')
('08439', '08439  CRIMMITSCHAU, STADT')
('06813', '06813  DESSAU-ROßLAU')
('01827', '01827  DUERRROEHRSDORF-DITTERSBACH')
('37547', '37547  EINBECK, STADT')
('04445', '04445  GROSSPOESNA')
('79683', '79683  KLEINES WIESENTAL')
('07558', '07558  KRAFTSDORF')
('31097', '31097  Lamspringe')
('08312', '08312  LAUTER-BERNSBACH, STADT')
('04007', '04007  LEIPZIG')
('09238', '09238  LICHTENAU')
('08146', '08146  LICHTENSTEIN/SA., STADT')
('08399', '08399  LIMBACH-OBERFROHNA, STADT')
('24875', '24875  MITTELANGELN')
('64743', '64743  Oberzent, Stadt')
('02744', '02744  ODERWITZ')
('04460', '04460  PEGAU, STADT')
('04657', '04657  PENIG, STADT')
('82243', '82243  POECKING')
('16921', '16921  PRITZWALK, STADT')
('04448', '04448  RACKWITZ')
('65423', '65423  RUESSELSHEI

# Concate Years (for NUTS3 and settlement types)
All steps above must already have been carried out for all years selected below.

In [14]:
years = [2030]
concate_focus_years = True

In [15]:
if concate_focus_years:
    X_nuts_profiles = {}
    X_siedlungstyp_profiles = {}
    X_nuts_pos_flex = {}
    X_siedlungstyp_pos_flex = {}
    X_nuts_neg_flex = {}
    X_siedlungstyp_neg_flex = {}
    X_nuts_driving = {}
    X_siedlungstyp_driving = {}

    for year in years:

        # all charging X_profiles 
        with open('data/nuts_profiles_{}.pickle'.format(year), 'rb') as f:
            nuts_profiles = pickle.load(f)
        with open('data/siedlungstyp_profiles_{}.pickle'.format(year), 'rb') as f:
            siedlungstyp_profiles = pickle.load(f)
        
        # pos flex X_profiles 
        with open('data/nuts_pos_flex_{}.pickle'.format(year), 'rb') as f:
            nuts_pos_flex = pickle.load(f)
        with open('data/siedlungstyp_pos_flex_{}.pickle'.format(year), 'rb') as f:
            siedlungstyp_pos_flex = pickle.load(f)

        # neg flex X_profiles 
        with open('data/nuts_neg_flex_{}.pickle'.format(year), 'rb') as f:
            nuts_neg_flex = pickle.load(f)
        with open('data/siedlungstyp_neg_flex_{}.pickle'.format(year), 'rb') as f:
            siedlungstyp_neg_flex = pickle.load(f)
        
        # driving X_profiles 
        with open('data/nuts_driving_{}.pickle'.format(year), 'rb') as f:
            nuts_driving= pickle.load(f)
        with open('data/siedlungstyp_driving_{}.pickle'.format(year), 'rb') as f:
            siedlungstyp_driving = pickle.load(f)

        # do it for all nuts3 regions
        """!!!"""
        for i in nuts_profiles:
            if i not in X_nuts_pos_flex.keys():
                X_nuts_profiles[i] = {}
                X_nuts_pos_flex[i] = {}
                X_nuts_neg_flex[i] = {}
                X_nuts_driving[i] = {}

            X_nuts_profiles[i][year]  = nuts_profiles[i][year]
            X_nuts_pos_flex[i][year]  = nuts_pos_flex[i][year]
            X_nuts_neg_flex[i][year]  = nuts_neg_flex[i][year]
            X_nuts_driving[i][year]  = nuts_driving[i][year]
            
        # do it for all settlement types
        """!!!"""
        for i in siedlungstyp_profiles:
            if i not in X_siedlungstyp_profiles.keys():
                X_siedlungstyp_profiles[i] = {}
                X_siedlungstyp_pos_flex[i] = {}
                X_siedlungstyp_neg_flex[i] = {}
                X_siedlungstyp_driving[i] = {}
                
            X_siedlungstyp_profiles[i][year]  = siedlungstyp_profiles[i][year]
            X_siedlungstyp_pos_flex[i][year]  = siedlungstyp_pos_flex[i][year]
            X_siedlungstyp_neg_flex[i][year]  = siedlungstyp_neg_flex[i][year]
            X_siedlungstyp_driving[i][year]  = siedlungstyp_driving[i][year]
    
    # transfer to MW for nuts3
    X_nuts_profiles  = {area:{year:{day:values/1000 for day,values in days.items()} for year,days in years.items()} for area,years in X_nuts_profiles.items()}
    X_nuts_pos_flex  = {area:{year:{day:values/1000 for day,values in days.items()} for year,days in years.items()} for area,years in X_nuts_pos_flex.items()}
    X_nuts_neg_flex  = {area:{year:{day:values/1000 for day,values in days.items()} for year,days in years.items()} for area,years in X_nuts_neg_flex.items()}
    X_nuts_driving  = {area:{year:{day:values/1000 for day,values in days.items()} for year,days in years.items()} for area,years in X_nuts_driving.items()}
    
    # transfer to MW for Siedlungstypen
    X_siedlungstyp_profiles  = {area:{year:{day:values/1000 for day,values in days.items()} for year,days in years.items()} for area,years in X_siedlungstyp_profiles.items()}
    X_siedlungstyp_pos_flex  = {area:{year:{day:values/1000 for day,values in days.items()} for year,days in years.items()} for area,years in X_siedlungstyp_pos_flex.items()}
    X_siedlungstyp_neg_flex  = {area:{year:{day:values/1000 for day,values in days.items()} for year,days in years.items()} for area,years in X_siedlungstyp_neg_flex.items()}
    X_siedlungstyp_driving  = {area:{year:{day:values/1000 for day,values in days.items()} for year,days in years.items()} for area,years in X_siedlungstyp_driving.items()}
    
    # Save for NUTS3
    with open('data/X_nuts_profiles.pickle', 'wb') as f:
        pickle.dump(X_nuts_profiles, f)
    with open('data/X_nuts_pos_flex.pickle', 'wb') as f:
        pickle.dump(X_nuts_pos_flex, f)
    with open('data/X_nuts_neg_flex.pickle', 'wb') as f:
        pickle.dump(X_nuts_neg_flex, f)
    with open('data/X_nuts_driving.pickle', 'wb') as f:
        pickle.dump(X_nuts_driving, f)
        
    # Save for settlement types
    with open('data/X_siedlungstyp_profiles.pickle', 'wb') as f:
        pickle.dump(X_siedlungstyp_profiles, f)
    with open('data/X_siedlungstyp_pos_flex.pickle', 'wb') as f:
        pickle.dump(X_siedlungstyp_pos_flex, f)
    with open('data/X_siedlungstyp_neg_flex.pickle', 'wb') as f:
        pickle.dump(X_siedlungstyp_neg_flex, f)
    with open('data/X_siedlungstyp_driving.pickle', 'wb') as f:
        pickle.dump(X_siedlungstyp_driving, f)
else:
    
    # Load for NUTS3
    with open('data/X_nuts_profiles.pickle', 'rb') as f:
        X_nuts_profiles = pickle.load(f)
    with open('data/X_nuts_pos_flex.pickle', 'rb') as f:
        X_nuts_pos_flex = pickle.load(f)
    with open('data/X_nuts_neg_flex.pickle', 'rb') as f:
        X_nuts_neg_flex = pickle.load(f)  
    with open('data/X_nuts_driving.pickle', 'rb') as f:
        X_nuts_driving = pickle.load(f)  
    
    # Load for NUTS3
    with open('data/X_siedlungstyp_profiles.pickle', 'rb') as f:
        X_siedlungstyp_profiles = pickle.load(f)
    with open('data/X_siedlungstyp_pos_flex.pickle', 'rb') as f:
        X_siedlungstyp_pos_flex = pickle.load(f)
    with open('data/X_siedlungstyp_neg_flex.pickle', 'rb') as f:
        X_siedlungstyp_neg_flex = pickle.load(f)  
    with open('data/X_siedlungstyp_driving.pickle', 'rb') as f:
        X_siedlungstyp_driving = pickle.load(f)  
